# FAST: the hydro-thermal problem

An implementation of the Hydro-thermal example from [FAST](https://github.com/leopoldcambier/FAST/tree/daea3d80a5ebb2c52f78670e34db56d53ca2e778/examples/hydro%20thermal)

In [ ]:
using SDDP, HiGHS, Test

function fast_hydro_thermal()
    model = SDDP.LinearPolicyGraph(;
        stages = 2,
        upper_bound = 0.0,
        sense = :Max,
        optimizer = HiGHS.Optimizer,
    ) do sp, t
        @variable(sp, 0 <= x <= 8, SDDP.State, initial_value = 0.0)
        @variables(sp, begin
            y >= 0
            p >= 0
            ξ
        end)
        @constraints(sp, begin
            p + y >= 6
            x.out <= x.in - y + ξ
        end)
        RAINFALL = (t == 1 ? [6] : [2, 10])
        SDDP.parameterize(sp, RAINFALL) do ω
            return JuMP.fix(ξ, ω)
        end
        @stageobjective(sp, -5 * p)
    end

    det = SDDP.deterministic_equivalent(model, HiGHS.Optimizer)
    set_silent(det)
    JuMP.optimize!(det)
    @test JuMP.objective_sense(det) == MOI.MAX_SENSE
    @test JuMP.objective_value(det) == -10
    SDDP.train(model)
    @test SDDP.calculate_bound(model) == -10
    return
end

fast_hydro_thermal()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*